In [ ]:
#setup - one-time tasks (run from terminal, not jupyter)
pip install cca-zoo

In [1]:
#import libraries
import pandas as pd
import numpy as np
import scipy
import os

userhome = os.path.expanduser('~')

In [2]:
# load residualized and scaled clinical features (from R script)
#discovery
clin_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/discovery/data/clin_features_scaled.csv'
#replication
#clin_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/replication/clin_features_scaled.csv'

clin_features = pd.read_csv(clin_file)
clin_features = clin_features.to_numpy()
print(type(clin_features))
print(clin_features.shape)

<class 'numpy.ndarray'>
(2846, 8)


In [4]:
# load residualized and scaled imaging features (from R script)
#discovery
rsFC_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/discovery/data/rsFC_features_scaled.csv'
#replication
#rsFC_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/replication/rsFC_features_scaled.csv'

rsFC_feat_chunks = pd.read_csv(rsFC_file, chunksize=1000)
rsFC_features = pd.concat(rsFC_feat_chunks)
#rsFC_features.sample(10)
rsFC_features = rsFC_features.to_numpy()
print(type(rsFC_features))
print(rsFC_features.shape)

<class 'numpy.ndarray'>
(2846, 61776)


In [5]:
# load selected rsFC features
#discovery
stable_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/discovery/results/discovery_stable_feats_95_Pearson05.csv'
#replication
#stable_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/replication/replication_stable_feats_95_Pearson05.csv'

stable_features = pd.read_csv(stable_features_file, header=None)
stable_features = stable_features.to_numpy()
stable_features = np.transpose(stable_features)
print(type(stable_features))
print(stable_features.shape)


<class 'numpy.ndarray'>
(1, 7095)


In [7]:
#Run elastic net CCA using full discovery sample and optimized hyperparameters
from cca_zoo.models import ElasticCCA
eCCA_model = ElasticCCA(max_iter=1000000000)

optim_c1 = 0.01
optim_c2 = 0.01
optim_l1_ratio = 0.75

#load selected feature indices file
allsubsample_selected_features = np.transpose(stable_features[:,:])
print(allsubsample_selected_features.shape)

#load selected feature indices file - alternative code block
#alldiscovery_selected_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/allDiscovery_selectedFeatsPearson.csv'
#alldiscovery_selected_features_file = userhome + r'/Desktop/OCS_bigdata_ML_local/data/allDiscovery_selectedFeatsSpearman.csv'
#alldiscovery_selected_features = pd.read_csv(alldiscovery_selected_features_file, header = None)
#alldiscovery_selected_features = alldiscovery_selected_features.to_numpy()
#print(alldiscovery_selected_features.shape)

allsubsample_rsFC_features = rsFC_features[:, allsubsample_selected_features[:,0] - 1]
#print(alldiscovery_rsFC_features)
print(allsubsample_rsFC_features.shape)

dims = min(np.linalg.matrix_rank(allsubsample_rsFC_features),np.linalg.matrix_rank(clin_features))
print('dims: ' + str(dims))

eCCA_fit = ElasticCCA(c = (optim_c1, optim_c2), l1_ratio = optim_l1_ratio, latent_dims=dims).fit([clin_features, allsubsample_rsFC_features])
print('params: ' + str(optim_c1) + ', ' + str(optim_c2) + ', ' + str(optim_l1_ratio))
allsubsample_scores = eCCA_fit.score((clin_features, allsubsample_rsFC_features))
print('All discovery CCA scores: ' + str(allsubsample_scores))

#calculate transformed canonical variate values
U,V = eCCA_fit.transform([clin_features, allsubsample_rsFC_features])
#U1,V1 = eCCA_fit.fit_transform([clin_features, alldiscovery_rsFC_features])



(7095, 1)
(2846, 7095)
dims: 8


/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 0 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 1 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 2 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 3 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 4 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/cca_zoo/models/iterative.py:106: UserWarning: Inner loop 5 not converged. Increase number of iterations.
  warnings.warn(
/opt/anaco

params: 0.01, 0.01, 0.75
All discovery CCA scores: [0.91598883 0.86340471 0.80823536 0.68006699 0.64355504 0.70307455
 0.69152796 0.62821598]


In [9]:
# Apply fitted enCCA model to test subset data
# load residualized and scaled clinical features (from R script)
clin_file_test = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/test/data/Data_prep/clin_features_scaled_testapplied.csv'
clin_features_test = pd.read_csv(clin_file_test)
clin_features_test = clin_features_test.to_numpy()
print(type(clin_features_test))
print(clin_features_test.shape)

# load residualized and scaled imaging features (from R script)
rsFC_file_test = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/test/data/Data_prep/rsFC_features_scaled_testapplied.csv'
rsFC_feat_chunks = pd.read_csv(rsFC_file_test, chunksize=1000)
rsFC_features_test = pd.concat(rsFC_feat_chunks)
#rsFC_features.sample(10)
rsFC_features_test = rsFC_features_test.to_numpy()
print(type(rsFC_features_test))
print(rsFC_features_test.shape)

# extract discovery-selected features
allsubsample_rsFC_features_test = rsFC_features_test[:, allsubsample_selected_features[:,0] - 1]
print(allsubsample_rsFC_features_test.shape)

#calculate transformed canonical variate values
U_test,V_test = eCCA_fit.transform([clin_features_test, allsubsample_rsFC_features_test])

print(U_test[:,0].shape)
print(V_test.shape)

allsubsample_scores_test = eCCA_fit.score((clin_features_test, allsubsample_rsFC_features_test))
print('All test CCA scores: ' + str(allsubsample_scores_test))

from scipy.stats.stats import pearsonr

#calculate correlation coefficient and p-value between x and y
print(pearsonr(U_test[:,0], V_test[:,0]))
print(pearsonr(U_test[:,1], V_test[:,1]))
print(pearsonr(U_test[:,2], V_test[:,2]))

<class 'numpy.ndarray'>
(145, 8)
<class 'numpy.ndarray'>
(145, 61776)
(145, 7095)
(145,)
(145, 8)
All test CCA scores: [ 0.12631146 -0.03670035 -0.04631702 -0.15386156  0.03673558  0.02580203
 -0.13806429  0.11877767]
(0.1263114551348102, 0.130051899827091)
(-0.03670034971251406, 0.6612025405800511)
(-0.04631701615371021, 0.580126087677948)


In [33]:
# Apply fitted enCCA model to replication subset data
# load residualized and scaled clinical features (from R script)
clin_file_replication = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/data/Data_prep/clin_features_scaled_replicationapplied.csv'
clin_features_replication = pd.read_csv(clin_file_replication)
clin_features_replication = clin_features_replication.to_numpy()
print(type(clin_features_replication))
print(clin_features_replication.shape)

# load residualized and scaled imaging features (from R script)
rsFC_file_replication = userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/data/Data_prep/rsFC_features_scaled_replicationapplied.csv'
rsFC_feat_chunks = pd.read_csv(rsFC_file_replication, chunksize=1000)
rsFC_features_replication = pd.concat(rsFC_feat_chunks)
rsFC_features_replication = rsFC_features_replication.to_numpy()
print(type(rsFC_features_replication))
print(rsFC_features_replication.shape)

# extract discovery-selected features
allsubsample_rsFC_features_replication = rsFC_features_replication[:, allsubsample_selected_features[:,0] - 1]
print(allsubsample_rsFC_features_replication.shape)

#calculate transformed canonical variate values
#U_replication,V_replication = eCCA_fit.transform([clin_features_replication, allsubsample_rsFC_features_replication])
#print(U_test[:,0].shape)
#print(V_test.shape)

allsubsample_scores_replication = eCCA_fit.score((clin_features_replication, allsubsample_rsFC_features_replication))
print('All replication CCA scores: ' + str(allsubsample_scores_replication))

#calculate transformed canonical variate values
U_replication,V_replication = eCCA_fit.transform([clin_features_replication, allsubsample_rsFC_features_replication])
print(pearsonr(U_replication[:,0], V_replication[:,0]))
print(pearsonr(U_replication[:,1], V_replication[:,1]))
print(pearsonr(U_replication[:,2], V_replication[:,2]))

<class 'numpy.ndarray'>
(2926, 8)
<class 'numpy.ndarray'>
(2926, 61776)
(2926, 7095)
All replication CCA scores: [ 3.74270432e-02  1.90390596e-02  2.55735662e-02 -2.86165654e-02
  4.28015142e-02 -5.33629071e-03  8.99221021e-05 -1.82293534e-02]
(0.037427043206763204, 0.042932403096607556)
(0.019039059613957778, 0.3032335811205344)
(0.02557356617744202, 0.16667297511677226)


In [22]:
# function to calculate CI for Pearson correlation from:
# https://gist.github.com/zhiyzuo/d38159a7c48b575af3e3de7501462e04
import numpy as np
from scipy import stats

def pearsonr_ci(x,y,alpha=0.05):
    ''' calculate Pearson correlation along with the confidence interval using scipy and numpy
    Parameters
    ----------
    x, y : iterable object such as a list or np.array
      Input for correlation calculation
    alpha : float
      Significance level. 0.05 by default
    Returns
    -------
    r : float
      Pearson's correlation coefficient
    pval : float
      The corresponding p value
    lo, hi : float
      The lower and upper bound of confidence intervals
    '''

    r, p = stats.pearsonr(x,y)
    r_z = np.arctanh(r)
    se = 1/np.sqrt(x.size-3)
    z = stats.norm.ppf(1-alpha/2)
    lo_z, hi_z = r_z-z*se, r_z+z*se
    lo, hi = np.tanh((lo_z, hi_z))
    return r, p, lo, hi

In [36]:
# calculate canonical correlations and CI for each subsample
#CV1
eCCA_canon_corrs1 = np.empty((3,4), dtype=object)
r_d, p_d, lo_d, hi_d = pearsonr_ci(U[:,0],V[:,0],alpha=0.05)
eCCA_canon_corrs1[0,:] = [r_d, p_d, lo_d, hi_d]
r_t, p_t, lo_t, hi_t = pearsonr_ci(U_test[:,0],V_test[:,0],alpha=0.05)
eCCA_canon_corrs1[1,:] = [r_t, p_t, lo_t, hi_t]
r_r, p_r, lo_r, hi_r = pearsonr_ci(U_replication[:,0],V_replication[:,0],alpha=0.05)
eCCA_canon_corrs1[2,:] = [r_r, p_r, lo_r, hi_r]

print(eCCA_canon_corrs1)
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allsubsets_canoncorrs1.csv', eCCA_canon_corrs1, delimiter=',')

#CV2
eCCA_canon_corrs2 = np.empty((3,4), dtype=object)
r_d, p_d, lo_d, hi_d = pearsonr_ci(U[:,1],V[:,1],alpha=0.05)
eCCA_canon_corrs2[0,:] = [r_d, p_d, lo_d, hi_d]
r_t, p_t, lo_t, hi_t = pearsonr_ci(U_test[:,1],V_test[:,1],alpha=0.05)
eCCA_canon_corrs2[1,:] = [r_t, p_t, lo_t, hi_t]
r_r, p_r, lo_r, hi_r = pearsonr_ci(U_replication[:,1],V_replication[:,1],alpha=0.05)
eCCA_canon_corrs2[2,:] = [r_r, p_r, lo_r, hi_r]

print(eCCA_canon_corrs2)
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allsubsets_canoncorrs2.csv', eCCA_canon_corrs2, delimiter=',')

#CV3
eCCA_canon_corrs3 = np.empty((3,4), dtype=object)
r_d, p_d, lo_d, hi_d = pearsonr_ci(U[:,2],V[:,2],alpha=0.05)
eCCA_canon_corrs3[0,:] = [r_d, p_d, lo_d, hi_d]
r_t, p_t, lo_t, hi_t = pearsonr_ci(U_test[:,2],V_test[:,2],alpha=0.05)
eCCA_canon_corrs3[1,:] = [r_t, p_t, lo_t, hi_t]
r_r, p_r, lo_r, hi_r = pearsonr_ci(U_replication[:,2],V_replication[:,2],alpha=0.05)
eCCA_canon_corrs3[2,:] = [r_r, p_r, lo_r, hi_r]

print(eCCA_canon_corrs3)
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/allsubsets_canoncorrs3.csv', eCCA_canon_corrs3, delimiter=',')



[[0.9159888259498956 0.0 0.9098686754442833 0.9217104376380338]
 [0.1263114551348102 0.130051899827091 -0.03746926543270223
  0.28348377407300857]
 [0.037427043206763204 0.042932403096607556 0.0011923901648641742
  0.07356354529758613]]
[[0.8634047068184905 0.0 0.8537462539762076 0.8724692056160552]
 [-0.03670034971251406 0.6612025405800511 -0.19852192403655802
  0.12706907136763385]
 [0.019039059613957778 0.3032335811205344 -0.01720908319117229
  0.05523722138695954]]
[[0.8082353618915168 0.0 0.7951049031016499 0.8206084600610438]
 [-0.04631701615371021 0.580126087677948 -0.20775765572878863
  0.11757995530252723]
 [0.02557356617744202 0.16667297511677226 -0.010672594196903943
  0.06175261079124772]]


In [7]:
# View clinical loadings
#alldiscovery_loadings = eCCA_fit.get_loadings((clin_features, alldiscovery_rsFC_features))
allsubsample_loadings = eCCA_fit.get_loadings((clin_features, allsubsample_rsFC_features), normalize=True)
allsubsample_loadings_u = allsubsample_loadings[0]
allsubsample_loadings_v = allsubsample_loadings[1]

print(allsubsample_loadings_u.shape)
print(allsubsample_loadings_v.shape)

print('All subsample - clinical loadings CV1: \n' + str(allsubsample_loadings_u[:,0]))
print('All subsample - clinical loadings CV2: \n' + str(allsubsample_loadings_u[:,1]))
print('All subsample - clinical loadings CV3: \n' + str(allsubsample_loadings_u[:,2]))
print('All subsample - clinical loadings CV4: \n' + str(allsubsample_loadings_u[:,3]))
print('All subsample - clinical loadings CV5: \n' + str(allsubsample_loadings_u[:,4]))
print('All subsample - clinical loadings CV6: \n' + str(allsubsample_loadings_u[:,5]))
print('All subsample - clinical loadings CV7: \n' + str(allsubsample_loadings_u[:,6]))
print('All subsample - clinical loadings CV8: \n' + str(allsubsample_loadings_u[:,7]))



(8, 8)
(7095, 8)
All subsample - clinical loadings CV1: 
[-29.87728398 -32.84983081 -17.7899728  -21.83395646 -33.11171109
 -32.24213927 -31.70820739 -30.56246484]
All subsample - clinical loadings CV2: 
[-19.50721211 -20.39291668 -45.32096311 -38.27131303  -5.98648856
   7.85647917  -0.66478871 -30.43564435]
All subsample - clinical loadings CV3: 
[-29.0733797   -0.80026369  -8.08636641  -8.75547724 -53.27694366
 -14.3327187    0.5671636  -25.27907334]
All subsample - clinical loadings CV4: 
[-0.90608016 66.27968297 16.16605231 20.17873163  6.16098722  5.11959725
  4.68256584 14.69063035]
All subsample - clinical loadings CV5: 
[-67.075833   -17.07174243   0.15018403 -16.27749895 -23.37626424
 -18.46033506 -18.69409442 -34.62252024]
All subsample - clinical loadings CV6: 
[18.83955419 19.36591198 14.86992767 39.82479859 19.54860999 53.70089834
  9.1889731  18.99078554]
All subsample - clinical loadings CV7: 
[-10.63135178 -15.92450723   7.74643613 -60.27605197  -6.28378588
  -5.457676

NameError: name 'subsample_loadings_u' is not defined

In [37]:
# Save clin loadings
#discovery
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV1norm.csv', alldiscovery_loadings_u[:,0], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV2norm.csv', alldiscovery_loadings_u[:,1], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV3norm.csv', alldiscovery_loadings_u[:,2], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV4norm.csv', alldiscovery_loadings_u[:,3], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV5norm.csv', alldiscovery_loadings_u[:,4], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV6norm.csv', alldiscovery_loadings_u[:,5], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV7norm.csv', alldiscovery_loadings_u[:,6], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_u_CV8norm.csv', alldiscovery_loadings_u[:,7], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_clinloadings_allnorm.csv', alldiscovery_loadings_u, delimiter=',')

#replication
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV1norm.csv', allsubsample_loadings_u[:,0], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV2norm.csv', allsubsample_loadings_u[:,1], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV3norm.csv', allsubsample_loadings_u[:,2], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV4norm.csv', allsubsample_loadings_u[:,3], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV5norm.csv', allsubsample_loadings_u[:,4], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV6norm.csv', allsubsample_loadings_u[:,5], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV7norm.csv', allsubsample_loadings_u[:,6], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_u_CV8norm.csv', allsubsample_loadings_u[:,7], delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_clinloadings_allnorm.csv', allsubsample_loadings_u, delimiter=',')


In [38]:
# View rs-FC loadings
allsubsample_loadings_v_CV1 = np.argsort(abs(allsubsample_loadings_v[:,0]))
#print(alldiscovery_loadings_v_CV1.shape)
print('CV1')
print(allsubsample_loadings_v_CV1[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV1[-20:],0])

allsubsample_loadings_v_CV2 = np.argsort(abs(allsubsample_loadings_v[:,1]))
print('CV2')
print(allsubsample_loadings_v_CV2[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV2[-20:],1])

allsubsample_loadings_v_CV3 = np.argsort(abs(allsubsample_loadings_v[:,2]))
print('CV3')
print(allsubsample_loadings_v_CV3[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV3[-20:],2])

allsubsample_loadings_v_CV4 = np.argsort(abs(allsubsample_loadings_v[:,3]))
print('CV4')
print(allsubsample_loadings_v_CV4[-20:] + 1)
print(allsubsample_loadings_v[allsubsample_loadings_v_CV4[-20:],3])

allsubsample_loadings_v_CV5 = np.argsort(abs(allsubsample_loadings_v[:,4]))
allsubsample_loadings_v_CV6 = np.argsort(abs(allsubsample_loadings_v[:,5]))
allsubsample_loadings_v_CV7 = np.argsort(abs(allsubsample_loadings_v[:,6]))
allsubsample_loadings_v_CV8 = np.argsort(abs(allsubsample_loadings_v[:,7]))

CV1
[4367  251 6159 6141 4726 2485 4327 1787 2592  838 1504 4775 3521 6711
 3674 2051  537  998 2593 1922]
[-4.09971767 -4.1593      4.20277735  4.24543649  4.28999037 -4.33883186
 -4.37614081 -4.38728231 -4.39512683 -4.40763811 -4.4174027  -4.46062148
 -4.47858484 -4.5664577  -4.69007376 -4.73655585 -4.75574114 -4.90351115
 -5.00129527 -5.25078755]
CV2
[4034 1421 2174 1174 1169 3774 3849 1404 6083 2227 2029 1167 1397 4295
 3093 4869  696 1393 6375 6617]
[ 3.55284146  3.59254275  3.59651959  3.60029606  3.61682154  3.64481171
 -3.65266786  3.66228956 -3.68816514  3.68996814  3.71382449  3.77947913
  3.77990887  3.82464273  3.87262592 -3.94564508 -4.00811307  4.09158324
  4.2842842   4.5273957 ]
CV3
[4888 5364 3653 1738  338 4947 1852 3641 4210 5865 4085 6388 1186 1847
 3768  295 5361 4886 1814   16]
[ 4.08360073  4.09179257 -4.10835447  4.12695245  4.17335466  4.18971925
  4.23528416 -4.23760067 -4.24451196 -4.2787938   4.30865052  4.37131098
  4.38472839  4.41652214  4.48455058 -4.651

In [39]:
# Save rsFC loadings
#discovery
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV1.csv', alldiscovery_loadings_v_CV1 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV1.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV1,0], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV2.csv', alldiscovery_loadings_v_CV2 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV2.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV2,1], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV3.csv', alldiscovery_loadings_v_CV3 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV3.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV3,2], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV4.csv', alldiscovery_loadings_v_CV4 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV4.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV4,3], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV5.csv', alldiscovery_loadings_v_CV5 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV5.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV5,4], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV6.csv', alldiscovery_loadings_v_CV6 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV6.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV6,5], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV7.csv', alldiscovery_loadings_v_CV7 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV7.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV7,6], delimiter=',')

#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCinds_v_CV8.csv', alldiscovery_loadings_v_CV8 + 1, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/alldiscovery_rsFCloadings_v_CV8.csv', alldiscovery_loadings_v[alldiscovery_loadings_v_CV8,7], delimiter=',')

#replication
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV1.csv', allsubsample_loadings_v_CV1 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV1.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV1,0], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV2.csv', allsubsample_loadings_v_CV2 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV2.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV2,1], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV3.csv', allsubsample_loadings_v_CV3 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV3.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV3,2], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV4.csv', allsubsample_loadings_v_CV4 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV4.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV4,3], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV5.csv', allsubsample_loadings_v_CV5 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV5.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV5,4], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV6.csv', allsubsample_loadings_v_CV6 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV6.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV6,5], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV7.csv', allsubsample_loadings_v_CV7 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV7.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV7,6], delimiter=',')

np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCinds_v_CV8.csv', allsubsample_loadings_v_CV8 + 1, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/replication/allreplication_rsFCloadings_v_CV8.csv', allsubsample_loadings_v[allsubsample_loadings_v_CV8,7], delimiter=',')

In [40]:
# Save files
#discovery
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/transformed_clin_U.csv', U, delimiter=',')
#np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/results/Pearson05_Stable95/transformed_rsFC_V.csv', V, delimiter=',')

#test
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/test/results/elasticnetCCA_scaled/applied_transformed_clin_U.csv', U_test, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/test/results/elasticnetCCA_scaled/applied_transformed_rsFC_V.csv', V_test, delimiter=',')

#replication
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/applied_transformed_clin_U.csv', U_replication, delimiter=',')
np.savetxt(userhome + r'/Desktop/OCS_bigdata_ML_local/FINAL/replication/results/elasticnetCCA_scaled/applied_transformed_rsFC_V.csv', V_replication, delimiter=',')


In [32]:
alldiscovery_scores = eCCA_fit.score((clin_features, alldiscovery_rsFC_features))
print('All discovery CCA scores: ' + str(alldiscovery_scores))



NameError: name 'alldiscovery_rsFC_features' is not defined